In [1]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_addons as tfa
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
print(tf.__version__)
AUTOTUNE = tf.data.experimental.AUTOTUNE

import deepcell
# Changed from before due to new placement of Track, concat_tracks
from deepcell_tracking.utils import load_trks
from deepcell.data.tracking import Track, concat_tracks
##############
from sklearn.model_selection import train_test_split
from deepcell.utils.data_utils import reshape_movie
from deepcell.utils.transform_utils import erode_edges
from deepcell.data import split_dataset
from deepcell_toolbox.processing import normalize, histogram_normalization

import spektral

2.8.0


In [2]:
# data loading

path_temp_trks = '/training/tracking-nuclear/train.trks'
all_data = [load_trks(path_temp_trks)]

In [3]:
%%time
all_tracks = [Track(tracked_data=d) for d in all_data]

100%|███████████████████████████████████████████| 91/91 [10:15<00:00,  6.77s/it]
2022-06-29 15:24:53.327243: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-29 15:24:54.706885: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10396 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0, compute capability: 6.1
2022-06-29 15:25:47.810607: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 5645002080 exceeds 10% of free system memory.


CPU times: user 1h 4min 21s, sys: 1h 16min 26s, total: 2h 20min 48s
Wall time: 19min


In [4]:
%%time
track_info = concat_tracks(all_tracks)

CPU times: user 3.25 s, sys: 4.57 s, total: 7.82 s
Wall time: 7.81 s


In [5]:
from skimage.exposure import rescale_intensity

def norm(X):
    X_norm = histogram_normalization(X, kernel_size=32)
    
    X_rescaled = np.zeros(X_norm.shape)
    for batch in range(X.shape[0]):
        for frame in range(X.shape[1]):
            x = X[batch, frame]
            x = rescale_intensity(x, out_range=(0,1))
            X_rescaled[batch, frame] = x
            
    return X_rescaled

In [6]:
for k, v in track_info.items():
    print(k, v.shape)

appearances (91, 71, 405, 32, 32, 1)
centroids (91, 71, 405, 2)
morphologies (91, 71, 405, 3)
adj_matrices (91, 71, 405, 405)
norm_adj_matrices (91, 71, 405, 405)
temporal_adj_matrices (91, 70, 405, 405, 3)


In [7]:
appearances = track_info['appearances']

app_shape = np.shape(appearances)
appearances = np.swapaxes(appearances, 1,2) #(0,2,1,3,4,5))
appearances = np.reshape(appearances, [-1, app_shape[1], app_shape[3], app_shape[4], app_shape[5]])

# Normalize appearances
appearances = norm(appearances)

input_dict = {'appearances': appearances}

output_dict = {'appearances_decoder': appearances}

data = tf.data.Dataset.from_tensor_slices((input_dict, output_dict))

data = data.repeat().batch(1)

2022-06-29 15:39:07.912285: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 21436047360 exceeds 10% of free system memory.
2022-06-29 15:39:29.861520: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 19.96GiB (rounded to 21436047360)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-06-29 15:39:29.861589: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-06-29 15:39:29.861624: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 3, Chunks in use: 3. 768B allocated for chunks. 768B in use in bin. 104B client-requested in use in bin.
2022-06-29 15:39:29.861638: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 0, Chun

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.